# Path 1 - Gemini API
This path will guide you on the set-up and usage of Google Gemini's API.

### 0. Get and store the API key
First of all, you need to login with your google account and get an API key [here](https://aistudio.google.com/app/apikey). It is **very important** that you do not share your API key with anyone and that you do not have it in your Repository.

You can keep your API key in a secure local document and access it when needed. It is common to save the key as an environmental variable so that it can be accessed by your python script.

However, this means your API key is in plain text in your script. To avoid this, if you're using VS Code, you can add your API key to a `.env` file in your workspace root with the following line:

```sh
GEMINI_API_KEY="PASTE YOUR KEY HERE"
```

Alternatively, you can use the [dot-env library](https://github.com/theskumar/python-dotenv).

In [2]:
# You can check if the environment variable API_KEY has been set up properly by running this line
!if [ -z $GOOGLE_API_KEY ]; then echo "\$GOOGLE_API_KEY not found"; else echo "\$GOOGLE_API_KEY found"; fi
!if [ -z $GEMINI_API_KEY ]; then echo "\$GEMINI_API_KEY not found"; else echo "\$GEMINI_API_KEY found"; fi

$GOOGLE_API_KEY found
$GEMINI_API_KEY found


### 1. First simple request
Now, you can write a simple script to see if everything is working properly.

In [2]:
from google import genai

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()  # here you can also pass the api_key directly using os.environ['GEMINI_API_KEY']

default_model = "gemini-2.5-flash"

client

#### Exercise 1

Ask the model to generate content about a random topic and print the response in text.

Here is the [official documentation](https://ai.google.dev/gemini-api/docs/text-generation?lang=python#configure) to find the help you need.

In [3]:
response = client.models.generate_content(
    model=default_model,
    contents="What is love?"
)
print(response.text)

Love is one of the most profound, complex, and deeply human experiences, incredibly difficult to capture in a single, definitive statement. It's universally felt yet uniquely experienced by each individual.

Here's an attempt to break down its multifaceted nature:

1.  **A Deep Emotion and Connection:**
    *   At its core, love is a strong feeling of affection, care, and attachment towards someone or something.
    *   It involves a sense of deep connection, belonging, and a bond that can transcend time and space.

2.  **A Set of Behaviors and Actions:**
    *   Love is not just a feeling; it's also a **choice** and a **verb**. It manifests through actions like:
        *   **Care and Support:** Looking out for another's well-being, offering help, being present.
        *   **Empathy and Understanding:** Trying to see the world through another's eyes, listening, validating feelings.
        *   **Commitment and Loyalty:** Standing by someone, even through difficulties, and showing fai

### 2. Generation parameters

When asking the model to generate some text, there are different parameters that you can tune to improve on the final quality of the text. [Here](https://ai.google.dev/gemini-api/docs/models/generative-models#model-parameters) is an overview of the parameters that Gemini offers. Try some of them in different context and understand how they affect the final generated text.

#### Exercise 2

Play with the output temperature, which controls the randomness of the generated text `temperature=0` means deterministic output, while `temperature=1` means maximum randomness (try some intermediate value too). Consider keeping the `max_output_tokens` to 50 so that the output is not too long; if you do, you should also set a low `thinking_budget` to avoid an empty response.

In [30]:
from google import genai
from google.genai import types

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="How does AI work?",
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0), # Disables thinking
        temperature=1,
        maxOutputTokens=50
    ),
)
print(response.text)


That's a fantastic question, and it's something many people are curious about! The term "AI" is a broad umbrella, so let's break down the core concepts and different ways it works.

At its heart, **AI


#### Exercise 3

Try out different `top_k` values, which controls how many tokens the model considers for output `top_k=1` means the model considers only one token for output (the one with the highest probability) `top_k=50` means the model considers the top 50 tokens for output.

In [38]:
from google import genai
from google.genai import types

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="How does AI work?",
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0), # Disables thinking
        temperature=0,
        maxOutputTokens=50,
        topK=50
    ),
)
print(response.text)

That's a fantastic question, and the answer is both incredibly complex and surprisingly simple at its core. Let's break down how AI works, starting with the basics and moving to more advanced concepts.

## The Core Idea: Learning from Data


#### Exercise 4

The same exercise as before but now with `top_p`, which controls how the model selects tokens for output `top_p=0.1` means the model selects tokens that make up 10% of the cumulative probability mass `top_p=0.9` means the model selects tokens that make up 90% of the cumulative probability mass `top_p` filters tokens *after* applying `top_k`.

Can you determine a rule of thumb as to how `top_k` and `top_p` affect the output results? (If you can't try to push the values to extreme values)

In [63]:
from google import genai
from google.genai import types

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="How does AI work?",
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0), # Disables thinking
        temperature=0.7,
        maxOutputTokens=50,
        topK=50,
        topP=0.1
    ),
)
print(response.text)

That's a fantastic question, and the answer is both complex and surprisingly simple at its core. Let's break down how AI works, starting with the basics and moving to more advanced concepts.

## The Core Idea: Learning from Data




Limiting the topK to 1 is like setting a low temperature because then it will only take the most likely token. The same goes for a low topP

### 3. Add images to the prompt

#### Exercise 5
Gemini, beside text also accepts images (and videos). Try prompting it with one. Choose an interesting image and prompt the model with a query about it.

You can use the [official documentation](https://ai.google.dev/gemini-api/docs/vision?lang=python#prompting-images).

Use [PIL](https://pillow.readthedocs.io/en/stable/) to load an image. It should already be present in the Python environment.

In [10]:
from PIL import Image
from IPython.display import display
from google import genai
from google.genai import types

IMAGE_PATH = "../data/engineer_fitting_prosthetic_arm.jpg"

# Your code here
im = Image.open(IMAGE_PATH)
#display(im)

client = genai.Client()

with open(IMAGE_PATH, 'rb') as f:
  image_bytes = f.read()

response = client.models.generate_content(
model='gemini-2.5-flash',
contents=[
  types.Part.from_bytes(
    data=image_bytes,
    mime_type='image/jpeg',
  ),
  'Caption this image.'
]
)

print(response.text)

Here are a few options for a caption, ranging in detail:

**Concise:**
*   Two men, both with advanced prosthetic limbs, collaborate on adjusting a bionic arm in a modern rehabilitation setting.

**Descriptive:**
*   In a bright, modern facility, a man with multiple limb prostheses, including a bionic arm, sits in a wheelchair while another man, also with a prosthetic arm, carefully adjusts the bionic limb. The scene suggests a fitting, training, or calibration session for advanced assistive technology.

**Detailed:**
*   A powerful image of advanced rehabilitation or research, showing one man in a wheelchair, fitted with bionic arms and leg prostheses, receiving focused assistance from another man, who also uses a sophisticated prosthetic arm. They are intently adjusting the bionic arm, likely during a fitting, programming, or training session for the cutting-edge device.

**Focus on Collaboration/Support:**
*   Embodying the spirit of innovation and peer support in prosthetics, two m

In [12]:
from PIL import Image
from IPython.display import display
from google import genai
from google.genai import types
import io

IMAGE_PATH = "../data/engineer_fitting_prosthetic_arm.jpg"

# Load image with PIL
im = Image.open(IMAGE_PATH)

# Optional: preprocess with PIL (resize, convert, etc.)
# im = im.convert("RGB")
# im = im.resize((512, 512))

# Convert PIL image into bytes in memory
buffer = io.BytesIO()
im.save(buffer, format="JPEG")   # or "PNG" depending on your needs
image_bytes = buffer.getvalue()

# Display in notebook (optional)
# display(im)

# Initialize client
client = genai.Client()

# Send to API
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=[
        types.Part.from_bytes(
            data=image_bytes,
            mime_type='image/jpeg',  # match the format you used in im.save()
        ),
        'Caption this image.'
    ]
)

print(response.text)

Here are a few caption options for the image, focusing on different aspects:

**Option 1 (Detailed & Comprehensive):**
In a bright, modern clinic or lab, a man in a wheelchair, equipped with advanced prosthetic arms and legs, has his upper limb prosthesis meticulously adjusted by another man. The assisting individual also appears to be wearing a prosthetic glove on his left hand, suggesting a shared understanding of prosthetic technology.

**Option 2 (Concise, highlighting collaboration):**
Two men, both users of advanced prosthetic technology, collaborate in a contemporary lab setting. One in a wheelchair has his bionic arm examined and adjusted by the other, who also has a visible prosthetic hand.

**Option 3 (Focus on rehabilitation/progress):**
A man in a wheelchair, with multiple advanced prosthetics, receives expert assistance with his bionic arm during a session in a modern rehabilitation facility. The person providing assistance also utilizes a prosthetic hand.

**Option 4 (Sim

### 4. Retrieval Augmented Generation (RAG)

#### Exercise 6

Depending on the application of the project, you might need to extract text from given documents and include it as additional context. This becomes especially relevant if you have many documents that cannot possibly fit into the model's context window. To more easily implement a RAG pipeline we recommend the use of one of these libraries: [LangChain](https://python.langchain.com/v0.2/docs/introduction/), [LlamaIndex](https://docs.llamaindex.ai/en/stable/examples/), [Haystack](https://docs.haystack.deepset.ai/docs/intro).

For the solution of this lab we will use *LangChain*.

It can be useful to split this exercise into these steps:
1. Read one or more documents using pdfminer
2. Split the documents into small chunks
3. Get and store the embeddings for each chunks
5. Given a query, retrieve the most relevant chunk(s) and appropriately prompt your LLM

**NOTE:** if you try to embed too many documents at once or too large documents you may run into rate limits. Possible solutions: 
* Reduce the number of chunks and/or their size
* Look at the HF version of this lab and use a local embedding model

In [7]:
import os  # langchain expects gemini's api key to be in the environment variable GOOGLE_API_KEY, use os to set it
import time
from langchain_google_genai import GoogleGenerativeAIEmbeddings  # get embeddings from Gemini
from langchain_community.vectorstores import FAISS  # "db" to store and retrieve embeddings
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter  # split long documents
import logging
# lower pdfminer logging so those messages don't appear
logging.getLogger("pdfminer").setLevel(logging.ERROR)
from pdfminer.high_level import extract_text  # extract text from pdfs

DOC_PATH = "../data/chain_of_thought_prompting.pdf"

# Suppose a user query
USER_QUERY = "What is CoT?"

# Load text
pdf_text = extract_text(DOC_PATH)
#print(pdf_text[:100])

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,     # Number of characters per chunk
    chunk_overlap=200,   # Overlap between chunks to preserve context
    length_function=len  # How chunk size is measured (default is len)
)

# Split the text into LangChain Document objects
chunks = text_splitter.split_documents(
    [Document(page_content=pdf_text, metadata={'source': DOC_PATH})]
)

'''
for i, chunk in enumerate(chunks[:3]):
    print(f'--- Chunk {i+1} ---')
    print(chunk.page_content[:300]) 
    print()
'''
print(f"Number of chunks: {len(chunks)}")

# Initialize Gemini embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create FAISS vectorstore from the chunks
#vectorstore = FAISS.from_documents(chunks, embeddings)

# Save the FAISS index so it can be reloaded later
#vectorstore.save_local("faiss_index")


Number of chunks: 30


E0000 00:00:1758832622.380524     185 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1758832622.387413     185 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [11]:
import os  # langchain expects gemini's api key to be in the environment variable GOOGLE_API_KEY, use os to set it
import time
from langchain_google_genai import GoogleGenerativeAIEmbeddings  # get embeddings from Gemini
from langchain_community.vectorstores import FAISS  # "db" to store and retrieve embeddings
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter  # split long documents
import logging
# lower pdfminer logging so those messages don't appear
logging.getLogger("pdfminer").setLevel(logging.ERROR)
from pdfminer.high_level import extract_text  # extract text from pdfs

DOC_PATH = "../data/chain_of_thought_prompting.pdf"

# Suppose a user query
USER_QUERY = "What is CoT?"

# Load text
pdf_text = extract_text(DOC_PATH)
#print(pdf_text[:100])

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,     # Number of characters per chunk
    chunk_overlap=200,   # Overlap between chunks to preserve context
    length_function=len  # How chunk size is measured (default is len)
)

# Split the text into LangChain Document objects
chunks = text_splitter.split_documents(
    [Document(page_content=pdf_text, metadata={'source': DOC_PATH})]
)

print(f"Number of chunks: {len(chunks)}")

# Initialize Gemini embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create FAISS vectorstore from the chunks
vectorstore = None  # start empty

# Batch the chunks
batch_size = 9 # free tier limit
for i in range(0, len(chunks), batch_size):
    batch = chunks[i:i + batch_size]

    # Embed and create FAISS index for this batch
    batch_vectorstore = FAISS.from_documents(batch, embeddings)

    if vectorstore is None:
        vectorstore = batch_vectorstore
    else:
        vectorstore.merge_from(batch_vectorstore)  # merge new batch into main index

    print(f'Processed batch {i // batch_size+1} / {(len(chunks) + batch_size - 1) // batch_size}')

    # only sleep if there is another batch
    if i + batch_size < len(chunks):
        print("Sleeping 65 seconds to respect rate limit...")
        time.sleep(65)


# Save the FAISS index so it can be reloaded later
vectorstore.save_local("faiss_index")
print("All chunks processed and stored in FAISS!")


Number of chunks: 30


E0000 00:00:1758833592.821245     185 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1758833592.824497     185 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


GoogleGenerativeAIError: Error embedding content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0 [violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerDayPerUserPerProjectPerModel-FreeTier"
}
violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerMinutePerUserPerProjectPerModel-FreeTie

### 5. Explore on your own
Gemini offers a bigger range of capabilities than those provided here, begin able to automatically handle multi-turn chats is one of them. Explore them on your own!

#### Exercise 7
Explore!

In [ ]:
# Your code here

### 6. Create a user interface

#### Exercise 8
Since you are trying to build a complete application, you also need a nice user interface that interacts with the model. There are various libraries available for this purpose. Notably: [gradio](https://www.gradio.app/docs/gradio/interface) and [chat UI](https://huggingface.co/docs/chat-ui/index). For the solution of this lab, we will use gradio.

Gradio has pre-defined input/output blocks that are automatically inserted in the interface. You only need to provide an appropriate function that takes all the inputs and returns the relevant output. See documentation [here](https://www.gradio.app/docs/gradio/interface).

Use a ChatInterface to create a chatbot UI that let's you discuss with Gemini, then add multimodal capabilities for both Gradio and Gemini.

In [ ]:
import gradio as gr

# This part closes the demo server if it is already running (which
# happens easily in notebooks) and prevents you from opening multiple
# servers at the same time.
if "demo" in locals() and demo.is_running:
    demo.close()

# Edit the parameters below
chats = {}  # store the chat history for each user (suppose multiple users)

# Your code here